In [12]:
# Exercise 1: Building a secure user authentication system
import bcrypt

class User:
    def __init__(self, username, password, priv_level = 'standard'):
        self.username = username
        self.password = self.hashed_password(password)
        self.priv_level = priv_level
        self.log_attempts = 0
        self.account_status = 'active'
        self.activity_log = []
        
    def hashed_password(self, password):
        hashed_pw = password
        s = bcrypt.gensalt()
        h = bcrypt.hashpw(hashed_pw, s)
        return h
    
    def authenticate(self, password):
        if self.account_status == "locked":
            self.log_activity("Login attempt on locked account")
            return False
        if self.hashed_password(password)==self.password:
            self.log_attempts = 0
            self.log_activity("You're successfully logged in.")
            return True

        else:
            self.log_attempts += 1
            self.log_activity(f"Failed login attempts: {self.log_attempts}")
            if self.log_attempts >= 3:
                self.lock_account()
            return False
        
 
    def check_privileges(self, required_level):
        privilege_hierarchy = {'guest': 0, 'standard': 1, 'admin': 2}
        return privilege_hierarchy.get(self.__privilege_level, 0) >= privilege_hierarchy.get(required_level, 0)
   
    def lock_account():
        self.account_status = "locked"
        self.log_activity("Account is locked due to failed login attempts")

    def reset_login_attempts(self, admin_password):  # Because admin is the only person who can reset the password
        if self.hashed_password(admin_password) == "hashed_admin_secret":
            self.account_status = "active"
            self.log_attempts = 0
            self.log_activity("Account unblocked by admin")
            return True
        return False
        
    def log_activity(self, message):
        from datetime import datetime
        self.activity_log.append(f"{datetime.now()}: {message}")

    def get_safe_info(self):
        return {
            "username":self.username,
            "priv_level": self.priv_level,
            "account_status": self.account_status
        }

    def get_username(self):
        return self.username

    def get_priv_level(self):
        return self.priv_level
        
        

user_1 = (f"'Alice'", "1234", "standard")
user_2 = ("'Bob'", "fnr44", "guest")
user_3 = ("'Cait'", "DC43", "admin")




In [ ]:
#Exercise 2: IoT Device Management System
class DeviceManager(self):
    self.list_of_devices = []

class Device(self, device_id, device_type, firmware_version, compliace_status, owner, last_security_scan, is_active):

    def authorise_access():
   
    def update_firmware():

    def run_security_scan():

    def check_compliance():
        #device must pass compliance checks before allowing access